In [1]:
from preamble import *

In [2]:
data = pd.read_csv('./data/j_data.csv')
print(type(data), data.shape)

<class 'pandas.core.frame.DataFrame'> (943412, 11)


In [3]:
data_dropped = data.drop(columns=['Date', 'output_flow'])
target = data['output_flow'].values.reshape(-1, 1)
data_dropped.head()

,input_flow,height_new,height_old,temperature,precipitation,wind_speed,humidity,atmospheric_pressure,snow
0,75.23,4.97,2.05,-0.9,0.0,1.4,72.0,1023.6,0.0
1,74.65,4.97,2.05,-0.9,0.0,1.4,72.0,1023.6,0.0
2,75.28,4.97,2.05,-0.9,0.0,1.4,72.0,1023.6,0.0
3,74.67,4.97,2.05,-0.9,0.0,1.4,72.0,1023.6,0.0
4,75.06,4.97,2.05,-0.9,0.0,1.4,72.0,1023.6,0.0


In [4]:
import torch
import torch.nn as nn

torch.__version__, torch.cuda.is_available()

('2.5.1', True)

In [6]:
train_size = int(data.shape[0] * 0.7)

trainset_feature = data_dropped[:train_size]
trainset_target = target[:train_size]
testset_feature = data_dropped[train_size:]
testset_target = target[train_size:]
print(trainset_feature.shape, testset_feature.shape)

(660388, 9) (283024, 9)


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler_feature = MinMaxScaler()
train_feature_scaled = scaler_feature.fit_transform(trainset_feature)
test_feature_scaled = scaler_feature.transform(testset_feature)
scaler_target = MinMaxScaler()
train_target_scaled = scaler_target.fit_transform(trainset_target)
test_target_scaled = scaler_target.transform(testset_target)

In [8]:
from tqdm import tqdm

def split_dataset(data, target, seq_len):
  X, y  = [], []
  for i in tqdm(range(data.shape[0]-(seq_len))):
    X.append(data[i:i+seq_len, :]) 
    y.append(target[i+seq_len])
  return np.array(X), np.array(y)

In [13]:
X_train, y_train = split_dataset(train_feature_scaled, train_target_scaled, 60)

  0%|          | 0/660328 [00:00<?, ?it/s]

100%|██████████| 660328/660328 [00:00<00:00, 1307408.31it/s]


In [14]:
X_train.shape, y_train.shape

((660328, 60, 9), (660328, 1))

In [15]:
X_test, y_test = split_dataset(test_feature_scaled, test_target_scaled, 60)

100%|██████████| 282964/282964 [00:00<00:00, 1291795.41it/s]


In [16]:
# y_train = y_train.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)

In [17]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((660328, 60, 9), (660328, 1), (282964, 60, 9), (282964, 1))

In [18]:
X_train_tensor, y_train_tensor = torch.tensor(X_train).float(), torch.tensor(y_train).float()
X_test_tensor, y_test_tensor = torch.tensor(X_test).float(), torch.tensor(y_test).float()

In [28]:
from torch.utils.data import DataLoader, TensorDataset

loader_train = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), shuffle=True,
                                           batch_size=64)
loader_test = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), shuffle=False,
                                          batch_size=64)

In [32]:
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(LSTMModel, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    lstm_out, _ = self.lstm(x)
    last_out = lstm_out[:, -1, :]
    out = self.fc(last_out)
    return out

input_size = 9
hidden_size = 50
output_size = 1
model = LSTMModel(input_size, hidden_size, output_size)


In [33]:
for I, label in loader_train:
  print(model(I)[0], label[0])
  break

tensor([0.0764], grad_fn=<SelectBackward0>) tensor([0.9576])


In [34]:
import torch.optim as optim

# loss_fn = torch.nn.MSELoss(reduction='mean')
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
num_epochs = 1
# loss_history = []
model.train()
model.to(device)
for epoch in range(num_epochs):
  running_loss = 0.0
  for X_batch, y_batch in loader_train:
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)

    out = model(X_batch)
    loss = loss_fn(out, y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  # loss_history.append(running_loss / len(loader_train.dataset))
  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(loader_train.dataset):.4f}")

Epoch [1/1], Loss: 0.0000
